In [4]:
# Imported Libraries

%matplotlib qt
#import yaml
#yaml.warnings({'YAMLLoadWarning': False})
# the default loader is deprecated, I dont know how to change the default loader
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import random
import pandas as pd
from matplotlib import colors

C:\Users\krish\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\krish\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
# Reading the files 

june = r"C:\Users\krish\Desktop\Courses\Thesis\data\era-iterim uvt\uvt_EI_daily_mean_1979-2018_06_prlev100-800_NHem.nc"
july = r"C:\Users\krish\Desktop\Courses\Thesis\data\era-iterim uvt\uvt_EI_daily_mean_1979-2018_07_prlev100-800_NHem.nc"
august = r"C:\Users\krish\Desktop\Courses\Thesis\data\era-iterim uvt\uvt_EI_daily_mean_1979-2018_08_prlev100-800_NHem.nc"


dataset_june = xr.open_dataset(june, chunks = {'time':10})
dataset_july = xr.open_dataset(july, chunks = {'time':10})
dataset_august = xr.open_dataset(august, chunks = {'time':10})

In [6]:
# Description of each dataset

#dataset_june.info()
#dataset_july.info()     
#dataset_august.info()
dataset_august

<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 121, level: 9, longitude: 480, time: 1240)
Coordinates:
  * time       (time) datetime64[ns] 1979-08-01T09:00:00 ... 2018-08-31T09:00:00
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.75 358.5 359.25
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... 2.25 1.5 0.75 0.0
  * level      (level) float64 100.0 150.0 200.0 250.0 ... 500.0 700.0 800.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<shape=(1240, 2), chunksize=(10, 2)>
    t          (time, level, latitude, longitude) float64 dask.array<shape=(1240, 9, 121, 480), chunksize=(10, 9, 121, 480)>
    u          (time, level, latitude, longitude) float64 dask.array<shape=(1240, 9, 121, 480), chunksize=(10, 9, 121, 480)>
    v          (time, level, latitude, longitude) float64 dask.array<shape=(1240, 9, 121, 480), chunksize=(10, 9, 121, 480)>
Attributes:
    CDI:          Climate Data Interface version 

In [7]:
dataset_august.level.values

array([100., 150., 200., 250., 300., 400., 500., 700., 800.])

In [8]:
dataset_june_pp = dataset_june['u'].sel(latitude = slice(90,0)).mean(dim = 'longitude')

In [9]:
dataset_june_pp

<xarray.DataArray 'u' (time: 1200, level: 9, latitude: 121)>
dask.array<shape=(1200, 9, 121), dtype=float64, chunksize=(10, 9, 121)>
Coordinates:
  * time      (time) datetime64[ns] 1979-06-01T09:00:00 ... 2018-06-30T09:00:00
  * latitude  (latitude) float32 90.0 89.25 88.5 87.75 ... 2.25 1.5 0.75 0.0
  * level     (level) float64 100.0 150.0 200.0 250.0 ... 500.0 700.0 800.0

In [10]:
dataset_june_pp.isel(time=35).plot(cmap = 'nipy_spectral', yincrease = False)

In [11]:
dataset_summer = xr.concat([dataset_june, dataset_july, dataset_august],dim = 'time').sortby('time')

In [37]:
dataset_JA = xr.concat([dataset_july, dataset_august],dim = 'time').sortby('time')

In [38]:
# Processing JA into SOM mode
dataset_JA_pp = dataset_JA.sel(latitude = slice(90,25)).mean(dim = 'longitude').sortby('latitude').sortby('level', ascending = False)
JA_data = dataset_JA_pp.u[:,:,:].values
JA_data_som = JA_data.reshape(JA_data.shape[0], -1)

In [12]:
dataset_summer

<xarray.Dataset>
Dimensions:    (bnds: 2, latitude: 121, level: 9, longitude: 480, time: 3680)
Coordinates:
  * longitude  (longitude) float32 0.0 0.75 1.5 2.25 ... 357.75 358.5 359.25
  * latitude   (latitude) float32 90.0 89.25 88.5 87.75 ... 2.25 1.5 0.75 0.0
  * level      (level) float64 100.0 150.0 200.0 250.0 ... 500.0 700.0 800.0
  * time       (time) datetime64[ns] 1979-06-01T09:00:00 ... 2018-08-31T09:00:00
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<shape=(3680, 2), chunksize=(10, 2)>
    t          (time, level, latitude, longitude) float64 dask.array<shape=(3680, 9, 121, 480), chunksize=(10, 9, 121, 480)>
    u          (time, level, latitude, longitude) float64 dask.array<shape=(3680, 9, 121, 480), chunksize=(10, 9, 121, 480)>
    v          (time, level, latitude, longitude) float64 dask.array<shape=(3680, 9, 121, 480), chunksize=(10, 9, 121, 480)>
Attributes:
    CDI:          Climate Data Interface version 

In [13]:
dataset_summer.level

<xarray.DataArray 'level' (level: 9)>
array([100., 150., 200., 250., 300., 400., 500., 700., 800.])
Coordinates:
  * level    (level) float64 100.0 150.0 200.0 250.0 ... 400.0 500.0 700.0 800.0
Attributes:
    standard_name:  air_pressure
    long_name:      pressure_level
    units:          millibars
    positive:       down
    axis:           Z

In [14]:
# This is only for u, the expression for getting v and t is different and don't have to averaged zonally
dataset_summer_pp = dataset_summer.sel(latitude = slice(90,25)).mean(dim = 'longitude').sortby('latitude').sortby('level', ascending = False)

In [15]:
dataset_summer_pp

<xarray.Dataset>
Dimensions:   (latitude: 87, level: 9, time: 3680)
Coordinates:
  * latitude  (latitude) float32 25.5 26.25 27.0 27.75 ... 87.75 88.5 89.25 90.0
  * level     (level) float64 800.0 700.0 500.0 400.0 ... 200.0 150.0 100.0
  * time      (time) datetime64[ns] 1979-06-01T09:00:00 ... 2018-08-31T09:00:00
Data variables:
    t         (time, level, latitude) float64 dask.array<shape=(3680, 9, 87), chunksize=(10, 9, 87)>
    u         (time, level, latitude) float64 dask.array<shape=(3680, 9, 87), chunksize=(10, 9, 87)>
    v         (time, level, latitude) float64 dask.array<shape=(3680, 9, 87), chunksize=(10, 9, 87)>

In [16]:
plt.figure()
dataset_summer_pp['u'].isel(time=97).plot(cmap = 'nipy_spectral', yincrease = False)

plt.figure()
dataset_summer_pp['t'].isel(time=97).plot(cmap = 'nipy_spectral', yincrease = False)

In [17]:
trial_data = dataset_summer_pp.u[0:92,:,:].values
whole_data = dataset_summer_pp.u[:,:,:].values

In [18]:
dates = dataset_summer_pp.time.values

In [19]:
plt.figure()
plt.plot_date(dates[5], 10)
plt.plot_date(dates[4], 5)
plt.plot_date(dates[6], 20)

In [20]:
#trial_data = np.moveaxis(trial_data, 0, -1)
#print(trial_data.shape)
print(whole_data.shape)
trial_data_som = trial_data.reshape(trial_data.shape[0],-1)
whole_data_som = whole_data.reshape(whole_data.shape[0], -1)
#trial_data_som = np.moveaxis(trial_data_som, 0, 1)
#print(len(trial_data_som.tolist()))
print(whole_data_som.shape)


(3680, 9, 87)
(3680, 783)


In [39]:
print(JA_data.shape)
print(JA_data_som.shape)

(2480, 9, 87)
(2480, 783)


In [21]:
# Carries all the centres from the clustering methos in the order of SOMS, KMeans, Hierarchical

centroids_6 = []

centroids_2 = []

centroids_4 = []

centroids_3 = []

In [22]:
# Iis used deflattening the array back 2 dimensions
lat = len(whole_data[1,2,:])
attr = len(whole_data_som[1,:])

year_2003 = np.arange(24*92,25*92)
year_2010 = np.arange(31*92,32*92)
year_2018 = np.arange(39*92,40*92)
all_years = np.arange(0,40*92)

In [23]:
from minisom import MiniSom

In [30]:
# 4 SOM
# Updated minimization of q error


# Main
iter = 10000
once = 1
soms = []
q_diffs = []
rands = []
# For older dataset
seed1 = 69
# For complete dataset
seed2018 = 8074
seed2010 = 628
seed2003 = 3233
seed_whole = 6240
seed_whole_10000 = 88248
seed_2003_10000 = 86374
seed_2010_10000 = 5755
seed_2018_10000 = 42155

for i in np.arange(once):
    #rand = np.random.randint(100000)
    #rands.append(rand)
    som4 = MiniSom(2,2,attr, sigma = 1, learning_rate = 0.5, random_seed = seed_whole_10000)
    som4.train_random(whole_data_som[:,:], 1000, verbose = False)
    soms.append(som4)
    q_diff = som4.quantization_error(whole_data_som[:,:])
    q_diffs.append(q_diff)
lowest = q_diffs.index(min(q_diffs))
som4 = soms[lowest]
#rand = rands[lowest]
print(min(q_diffs))
#print(rand)

82.95196889947069


In [382]:
sorted_rands = rands[:]
sorted_rands.sort()
corresponding_q_diffs = []
for rand in sorted_rands:
    i = rands.index(rand)
    corresponding_q_diffs.append(q_diffs[i])
plt.figure()
plt.title('Variation of quantization errors')
plt.plot(sorted_rands,corresponding_q_diffs)
plt.xlabel("Generated random number")
plt.ylabel("Quantization error")
plt.show()

In [348]:
# Figuring out the function to relate each point in the sample to a dataset

#print(som4.quantization(whole_data_som))
print(len(som4.win_map(whole_data_som).keys()))
#for data in whole_data_som:
   # print(som4.winner(data))

4
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 1)
(0, 1)
(0, 0)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 

(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)

(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 0)
(0, 1)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(1, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(1, 0)
(0, 1)
(1, 0)
(1, 0)
(1, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)
(0, 1)

In [135]:
# Data that is to be plotted

trial_plot_node_1 = np.reshape(som4.get_weights()[0,0,:],(-1,lat))
trial_plot_node_2 = np.reshape(som4.get_weights()[0,1,:],(-1,lat))
trial_plot_node_3 = np.reshape(som4.get_weights()[1,0,:],(-1,lat))
trial_plot_node_4 = np.reshape(som4.get_weights()[1,1,:],(-1,lat))
centroids_4.extend((som4.get_weights()[0,0,:], som4.get_weights()[0,1,:], som4.get_weights()[1,0,:], som4.get_weights()[1,1,:]))

In [136]:
np.asarray(centroids_4).shape

(16, 783)

In [138]:
# Plotting the nodes
plt.figure()
plt.suptitle("2 by 2 SOM-1979-2018 with seed")
plt.subplot(2,2,1)
plt.title("Freq --> %.2f%%" %(100*len(som4.win_map(whole_data_som[:,:])[(0,0)])/len(whole_data_som[:,1])))
plt.imshow(trial_plot_node_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.gca().invert_xaxis()
#plt.gca().invert_yaxis()
#plt.xticks(np.arange(25,91,7.5))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.xlabel("Latitude(degrees North)")
plt.ylabel("Pressure Levels")
plt.colorbar()
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.subplot(2,2,2)
plt.title("Freq --> %.2f%%" %(100*len(som4.win_map(whole_data_som[:,:])[(0,1)])/len(whole_data_som[:,1])))
plt.imshow(trial_plot_node_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.gca().invert_xaxis()
#plt.gca().invert_yaxis()
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))

plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,3)
plt.title("Freq --> %.2f%%" %(100*len(som4.win_map(whole_data_som[:,:])[(1,0)])/len(whole_data_som[:,1])))
plt.imshow(trial_plot_node_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.gca().invert_xaxis()
#plt.gca().invert_yaxis()
#plt.xticks(np.arange(0,45,5), year_2003p.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,4)
plt.title("Freq --> %.2f%%" %(100*len(som4.win_map(whole_data_som[:,:])[(1,1)])/len(whole_data_som[:,1])))
plt.imshow(trial_plot_node_4, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.gca().invert_xaxis()
#plt.gca().invert_yaxis()
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()


In [222]:
#2 SOM
# Updated minimization of q error


# main
iter = 10000
once = 1
soms = []
q_diffs = []
rands = []
seed = 6792
seed_whole_10000 = 26395
seed_2003_10000 = 47314
seed_2010_10000 = 81137
seed_2018_10000 = 10583
for i in np.arange(once):
    #rand = np.random.randint(100000)
    #rands.append(rand)
    som2 = MiniSom(2,1,attr, sigma = 0.5, learning_rate = 0.5, random_seed = seed_whole_10000)
    som2.train_random(whole_data_som[:,:], 1000, verbose = False)
    soms.append(som2)
    q_diff = som2.quantization_error(whole_data_som[:,:])
    #print(q_diff)
    q_diffs.append(q_diff)
lowest = q_diffs.index(min(q_diffs))
#seed = rands[lowest]
som2 = soms[lowest]
print(min(q_diffs))
#print(seed)
#print("\n")
#print(som2.get_weights().shape)

92.49836027560264


In [228]:
time_series(som2, time_interval = 92)

C1uster_1-yearly segregation
Years with high freq of double jet events with corresponding frequencies
{1979: 49, 1980: 43, 1981: 53, 1982: 46, 1983: 43, 1985: 63, 1987: 61, 1988: 51, 1990: 52, 1991: 38, 1993: 57, 1995: 61, 1996: 37, 1997: 47, 1998: 46, 1999: 50, 2000: 43, 2001: 38, 2002: 43, 2003: 43, 2004: 42, 2005: 50, 2007: 59, 2008: 51, 2009: 67, 2010: 42, 2011: 71, 2012: 42, 2014: 46, 2015: 74, 2018: 42}
Years with medium freq of double jet events with corresponding frequencies
{1984: 33, 1986: 31, 1992: 27, 2013: 25, 2016: 26, 2017: 27}
Years with low freq of double jet events with corresponding frequencies
{1989: 18, 1994: 23, 2006: 19}
Cluster_2-yearly segregation
Years with high freq of double jet events with corresponding frequencies
{1979: 43, 1980: 49, 1981: 39, 1982: 46, 1983: 49, 1984: 59, 1986: 61, 1988: 41, 1989: 74, 1990: 40, 1991: 54, 1992: 65, 1993: 35, 1994: 69, 1996: 55, 1997: 45, 1998: 46, 1999: 42, 2000: 49, 2001: 54, 2002: 49, 2003: 49, 2004: 50, 2005: 42, 2006:

In [177]:
# Data to be plotted
trial_plot_node_1 = np.reshape(som2.get_weights()[0,0,:],(-1,lat))
trial_plot_node_2 = np.reshape(som2.get_weights()[1,0,:],(-1,lat))
centroids_2.extend((som2.get_weights()[0,0,:], som2.get_weights()[1,0,:]))

In [172]:
# Plotting the nodes
plt.suptitle("2 by 1 SOM")
plt.subplot(2,1,1)
plt.title("Freq --> %.2f%%" %(100*len(som2.win_map(whole_data_som[:,:])[(0,0)])/len(whole_data_som[:,1])))
plt.imshow(trial_plot_node_1, vmin = -10, vmax = 25, interpolation = 'spline36', aspect = 'auto', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,1,2)
plt.title("Freq --> %.2f%%" %(100*len(som2.win_map(whole_data_som[:,:])[(1,0)])/len(whole_data_som[:,1])))
plt.imshow(trial_plot_node_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [ ]:
#3 SOM
# Updated minimization of q error


# main

iter = 10000
once = 1
soms = []
q_diffs = []
rands = []
seed = 6792

seed_JA_10000 = 0
seed_whole_10000 = 16408
seed_2003_10000 = 80361
seed_2010_10000 = 25039
seed_2018_10000 = 38534
for i in np.arange(iter):
    rand = np.random.randint(100000)
    rands.append(rand)
    som3 = MiniSom(3,1,attr, sigma = 0.5, learning_rate = 0.5, random_seed = rand)
    som3.train_random(JA_data_som[:,:], 1000, verbose = False)
    soms.append(som3)
    q_diff = som3.quantization_error(JA_data_som[:,:])
    #print(q_diff)
    q_diffs.append(q_diff)
lowest = q_diffs.index(min(q_diffs))
seed = rands[lowest]
som3 = soms[lowest]
print(min(q_diffs))
#print(seed)
#print("\n")
#print(som2.get_weights().shape)

In [ ]:
# Data to be plotted
trial_plot_node_1 = np.reshape(som3.get_weights()[0,0,:],(-1,lat))
trial_plot_node_2 = np.reshape(som3.get_weights()[1,0,:],(-1,lat))
trial_plot_node_3 = np.reshape(som3.get_weights()[2,0,:],(-1,lat))
centroids_3.extend((som3.get_weights()[0,0,:], som3.get_weights()[1,0,:], som3.get_weights()[2,0,:]))

In [ ]:
seed

In [ ]:
# Plotting the nodes
plt.figure()
plt.suptitle("3 by 1 SOM - 1979-2018 (July and August)")
plt.subplot(3,1,1)
plt.title("Freq --> %.2f%%" %(100*len(som3.win_map(JA_data_som[:,:])[(0,0)])/len(JA_data_som[:,1])))
plt.imshow(trial_plot_node_1, vmin = -10, vmax = 25, interpolation = 'spline36', aspect = 'auto', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,1,2)
plt.title("Freq --> %.2f%%" %(100*len(som3.win_map(JA_data_som[:,:])[(1,0)])/len(JA_data_som[:,1])))
plt.imshow(trial_plot_node_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,1,3)
plt.title("Freq --> %.2f%%" %(100*len(som3.win_map(JA_data_som[:,:])[(2,0)])/len(JA_data_som[:,1])))
plt.imshow(trial_plot_node_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [253]:
#6 SOM

# Main
iter = 10000
once = 1
soms = []
q_diffs = []
rands = []
seed = 5036
seed_whole_10000 = 1067
seed_2003_10000 = 54417
seed_2010_10000 = 37365
seed_2018_10000 = 35225
seed_2003 = 4121
year_2003 = np.arange(24*92,25*92)
year_2010 = np.arange(31*92,32*92)
year_2018 = np.arange(39*92,40*92)
for i in np.arange(once):
    #rand = np.random.randint(100000)
    #rands.append(rand)
    som = MiniSom(3,2,783, sigma = 1, learning_rate = 0.5, random_seed = seed_2018_10000)
    som.train_random(whole_data_som[year_2018,:], 1000, verbose = False)
    soms.append(som)
    q_diff = som.quantization_error(whole_data_som[year_2018,:])
    q_diffs.append(q_diff)

lowest = q_diffs.index(min(q_diffs))
som6 = soms[lowest]
#rand = rands[lowest]
print(min(q_diffs))
#print(rand)
#print("\n")
#print(som.get_weights().shape)

69.36494805113753


In [112]:
# Debugging each event frequency

print(len(som6.win_map(whole_data_som[year_2018,:])[(1,1)]))

10


In [113]:
# Data to be plotted
trial_plot_node_1 = np.reshape(som6.get_weights()[0,0,:],(-1,lat))
trial_plot_node_2 = np.reshape(som6.get_weights()[0,1,:],(-1,lat))
trial_plot_node_3 = np.reshape(som6.get_weights()[1,0,:],(-1,lat))
trial_plot_node_4 = np.reshape(som6.get_weights()[1,1,:],(-1,lat))
trial_plot_node_5 = np.reshape(som6.get_weights()[2,0,:],(-1,lat))
trial_plot_node_6 = np.reshape(som6.get_weights()[2,1,:],(-1,lat))
centroids_6.extend((som6.get_weights()[0,0,:], som6.get_weights()[0,1,:], som6.get_weights()[1,0,:], som6.get_weights()[1,1,:], som6.get_weights()[2,0,:], som6.get_weights()[2,1,:]))

In [429]:
# PLotting

# Plotting the nodes
plt.figure()
plt.suptitle("3 by 2 SOM 2018 with seed")
plt.subplot(3,2,1)
plt.title("Freq --> %.2f%%" %(100*len(som6.win_map(whole_data_som[year_2018,:])[(0,0)])/len(whole_data_som[year_2018,1])))
plt.imshow(trial_plot_node_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,2)
plt.title("Freq --> %.2f%%" %(100*len(som6.win_map(whole_data_som[year_2018,:])[(0,1)])/len(whole_data_som[year_2018,1])))
plt.imshow(trial_plot_node_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,3)
plt.title("Freq --> %.2f%%" %(100*len(som6.win_map(whole_data_som[year_2018,:])[(1,0)])/len(whole_data_som[year_2018,1])))
plt.imshow(trial_plot_node_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,4)
plt.title("Freq --> %.2f%%" %(100*len(som6.win_map(whole_data_som[year_2018,:])[(1,1)])/len(whole_data_som[year_2018,1])))
plt.imshow(trial_plot_node_4, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,5)
plt.title("Freq --> %.2f%%" %(100*len(som6.win_map(whole_data_som[year_2018,:])[(2,0)])/len(whole_data_som[year_2018,1])))
plt.imshow(trial_plot_node_5, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,6)
plt.title("Freq --> %.2f%%" %(100*len(som6.win_map(whole_data_som[year_2018,:])[(2,1)])/len(whole_data_som[year_2018,1])))
plt.imshow(trial_plot_node_6, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [88]:
from sklearn.cluster import KMeans
from collections import Counter

In [26]:
# Trying k means clustering.

#from sklearn.cluster import Kmeans
#import sklearn

kmeans2 = KMeans(n_clusters = 2, n_init = 100, max_iter = 1000, random_state = 0).fit(whole_data_som[:,:])
centres2 = kmeans2.cluster_centers_

In [27]:
freq2 = Counter(kmeans2.labels_)

In [28]:
#2 centres K means clustering

centroids_2.extend((centres2[0], centres2[1]))
centre_1 = np.reshape(centres2[0],(-1,lat))
centre_2 = np.reshape(centres2[1],(-1,lat))

In [29]:
# Plotting the nodes
plt.suptitle("2 Kmeans")
plt.subplot(2,1,1)
plt.title("Freq --> %.2f%%" %(100*freq2[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,1,2)
plt.title("Freq --> %.2f%%" %(100*freq2[1]/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

C:\Users\krish\Anaconda3\lib\site-packages\matplotlib\figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [462]:
# 3 centres K means clustering


kmeans3 = KMeans(n_clusters = 3, n_init = 100, max_iter = 1000, random_state = 5).fit(whole_data_som[:,:])
centres3 = kmeans3.cluster_centers_
centroids_3.extend((centres3[0], centres3[1], centres3[2]))

In [464]:
kmeans3.predict(whole_data_som[:,:])

array([0, 0, 0, ..., 1, 1, 1])

In [203]:
centre_1 = np.reshape(centres3[0],(-1,lat))
centre_2 = np.reshape(centres3[1],(-1,lat))
centre_3 = np.reshape(centres3[2],(-1,lat))
freq3 = Counter(kmeans3.labels_)

In [205]:
# Plotting the nodes
plt.figure()
plt.suptitle("3 Kmeans 1979-2018 JJA")
plt.subplot(3,1,2)
plt.title("Freq --> %.2f%%" %(100*freq3[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,1,1)
plt.title("Freq --> %.2f%%" %(100*freq3[1]/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,1,3)
plt.title("Freq --> %.2f%%" %(100*freq3[2]/len(whole_data_som[:,1])))
plt.imshow(centre_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [74]:
# 4 centres K means clustering


kmeans4 = KMeans(n_clusters = 4, n_init = 100, max_iter = 1000, random_state = 5).fit(whole_data_som[year_2018,:])
centres4 = kmeans4.cluster_centers_
centroids_4.extend((centres4[0], centres4[1], centres4[2], centres4[3]))

In [325]:
centre_1 = np.reshape(centres4[0],(-1,lat))
centre_2 = np.reshape(centres4[1],(-1,lat))
centre_3 = np.reshape(centres4[2],(-1,lat))
centre_4 = np.reshape(centres4[3],(-1,lat))
freq4 = Counter(kmeans4.labels_)

In [326]:
# Plotting the nodes
plt.figure()
plt.suptitle("4 Kmeans 1979-2018")
plt.subplot(2,2,1)
plt.title("Freq --> %.2f%%" %(100*freq4[2]/len(whole_data_som[:,1])))
plt.imshow(centre_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,2)
plt.title("Freq --> %.2f%%" %(100*freq4[1]/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,3)
plt.title("Freq --> %.2f%%" %(100*freq4[3]/len(whole_data_som[:,1])))
plt.imshow(centre_4, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,4)
plt.title("Freq --> %.2f%%" %(100*freq4[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [114]:
# 6 centres K means clustering

kmeans6 = KMeans(n_clusters = 6, n_init = 100, max_iter = 1000, random_state = 5).fit(whole_data_som[year_2018,:])
centres6 = kmeans6.cluster_centers_

In [115]:
centroids_6.extend((centres6[0], centres6[1], centres6[2], centres6[3], centres6[4], centres6[5]))

In [90]:
centre_1 = np.reshape(centres6[0],(-1,44))
centre_2 = np.reshape(centres6[1],(-1,44))
centre_3 = np.reshape(centres6[2],(-1,44))
centre_4 = np.reshape(centres6[3],(-1,44))
centre_5 = np.reshape(centres6[4],(-1,44))
centre_6 = np.reshape(centres6[5],(-1,44))

In [91]:
freq6 = Counter(kmeans6.labels_)
# Plotting the nodes
plt.suptitle("6 Kmeans")
plt.subplot(3,2,1)
plt.title("Freq --> %.2f%%" %(100*freq6[3]/len(whole_data_som[:,1])))
plt.imshow(centre_4, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,2)
plt.title("Freq --> %.2f%%" %(100*freq6[5]/len(whole_data_som[:,1])))
plt.imshow(centre_6, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,3)
plt.title("Freq --> %.2f%%" %(100*freq6[1]/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,4)
plt.title("Freq --> %.2f%%" %(100*freq6[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,5)
plt.title("Freq --> %.2f%%" %(100*freq6[2]/len(whole_data_som[:,1])))
plt.imshow(centre_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,6)
plt.title("Freq --> %.2f%%" %(100*freq6[4]/len(whole_data_som[:,1])))
plt.imshow(centre_5, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [83]:
from sklearn.cluster import AgglomerativeClustering

In [48]:
cluster = AgglomerativeClustering(n_clusters = 2, linkage = "ward").fit(whole_data_som[:,:])

In [49]:
centres_1 = []
centres_2 = []
for i  in np.arange(len(cluster.labels_)):
    #k = year_2003[i]
    #k = year_2010[i]
    #k = year_2018[i]
    #k = i
    if cluster.labels_[i]==0:
        centres_1.append(whole_data_som[i,:])
    else:
        centres_2.append(whole_data_som[i,:])
centres2_1 = np.mean(centres_1, axis=0)
centres2_2 = np.mean(centres_2, axis=0)

In [50]:
centroids_2.extend((centres2_1, centres2_2))

In [80]:
centre_1 = np.reshape(centres2_1,(-1,lat))
centre_2 = np.reshape(centres2_2,(-1,lat))

In [81]:
freq_h2 = Counter(cluster.labels_)

In [82]:
# Plotting the nodes
plt.suptitle("Hierarchical n = 2")
plt.subplot(2,1,1)
plt.title("Freq --> %.2f%%" %((100*freq_h2[1])/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,1,2)
plt.title("Freq --> %.2f%%" %(100*freq_h2[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [50]:
year_2003

array([2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215, 2216, 2217, 2218,
       2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229,
       2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240,
       2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251,
       2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262,
       2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273,
       2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284,
       2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295,
       2296, 2297, 2298, 2299])

In [469]:
# 3 centres Hierarchical

cluster3 = AgglomerativeClustering(n_clusters = 3, linkage = "ward").fit(whole_data_som[:,:])

centres_1 = []
centres_2 = []
centres_3 = []

for i  in np.arange(len(cluster3.labels_)):
    #k = year_2003[i]
    #k = year_2010[i]
    #k = year_2018[i]
    k = i
    if cluster3.labels_[i]==0:
        centres_1.append(whole_data_som[k,:])
    elif cluster3.labels_[i]==1:
        centres_2.append(whole_data_som[k,:])
    else:
        centres_3.append(whole_data_som[k,:])
    
centres3_1 = np.mean(centres_1, axis=0)
centres3_2 = np.mean(centres_2, axis=0)
centres3_3 = np.mean(centres_3, axis=0)


In [188]:
centroids_3.extend((centres3_1, centres3_2, centres3_3))

In [189]:
centre_1 = np.reshape(centres3_1,(-1,lat))
centre_2 = np.reshape(centres3_2,(-1,lat))
centre_3 = np.reshape(centres3_3,(-1,lat))

In [190]:
freq_h3 = Counter(cluster3.labels_)

In [339]:
# Plotting the nodes
plt.figure()
plt.suptitle("Hierarchical n = 3 1979-2018")
plt.subplot(3,1,2)
plt.title("Freq --> %.2f%%" %(100*freq_h3[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,1,1)
plt.title("Freq --> %.2f%%" %(100*freq_h3[1]/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,1,3)
plt.title("Freq --> %.2f%%" %(100*freq_h3[2]/len(whole_data_som[:,1])))
plt.imshow(centre_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [75]:
# 4 centres Hierarchical

cluster4 = AgglomerativeClustering(n_clusters = 4, linkage = "ward").fit(whole_data_som[:,:])

centres_1 = []
centres_2 = []
centres_3 = []
centres_4 = []


for i  in np.arange(len(cluster4.labels_)):
    #k = year_2003[i]
    #k = year_2010[i]
    #k = year_2018[i]
    k = i
    if cluster4.labels_[i]==0:
        centres_1.append(whole_data_som[k,:])
    elif cluster4.labels_[i]==1:
        centres_2.append(whole_data_som[k,:])
    elif cluster4.labels_[i]==2:
        centres_3.append(whole_data_som[k,:])
    else:
        centres_4.append(whole_data_som[k,:])

centres4_1 = np.mean(centres_1, axis=0)
centres4_2 = np.mean(centres_2, axis=0)
centres4_3 = np.mean(centres_3, axis=0)
centres4_4 = np.mean(centres_4, axis=0)

In [76]:
centroids_4.extend((centres4_1, centres4_2, centres4_3, centres4_4))

In [53]:
centre_1 = np.reshape(centres4_1,(-1,lat))
centre_2 = np.reshape(centres4_2,(-1,lat))
centre_3 = np.reshape(centres4_3,(-1,lat))
centre_4 = np.reshape(centres4_4,(-1,lat))

In [54]:
freq_h4 = Counter(cluster4.labels_)

In [55]:
print(freq_h4)

Counter({0: 30, 1: 23, 2: 22, 3: 17})


In [332]:
# Plotting the nodes
plt.figure()
plt.suptitle("Hierarchical n = 4 1979-2018")
plt.subplot(2,2,1)
plt.title("Freq --> %.2f%%" %(100*freq_h4[2]/len(whole_data_som[:,1])))
plt.imshow(centre_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,2)
plt.title("Freq --> %.2f%%" %(100*freq_h4[1]/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,3)
plt.title("Freq --> %.2f%%" %(100*freq_h4[3]/len(whole_data_som[:,1])))
plt.imshow(centre_4, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(2,2,4)
plt.title("Freq --> %.2f%%" %(100*freq_h4[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
#plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
#plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.xticks(np.arange(0,lat+1,10), np.arange(25,91,7.5))
plt.yticks(np.arange(-0.5,8.5,1), reversed(np.arange(100,801,70)))
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [116]:
# 6 centres Hierarchical

cluster6 = AgglomerativeClustering(n_clusters = 6, linkage = "ward").fit(whole_data_som[year_2018,:])

centres_1 = []
centres_2 = []
centres_3 = []
centres_4 = []
centres_5 = []
centres_6 = []

for i  in np.arange(len(cluster6.labels_)):
    #k = year_2003[i]
    #k = year_2010[i]
    k = year_2018[i]
    #k = i
    if cluster6.labels_[i]==0:
        centres_1.append(whole_data_som[k,:])
    elif cluster6.labels_[i]==1:
        centres_2.append(whole_data_som[k,:])
    elif cluster6.labels_[i]==2:
        centres_3.append(whole_data_som[k,:])
    elif cluster6.labels_[i]==3:
        centres_4.append(whole_data_som[k,:])
    elif cluster6.labels_[i]==4:
        centres_5.append(whole_data_som[k,:])
    else:
        centres_6.append(whole_data_som[k,:])

centres6_1 = np.mean(centres_1, axis=0)
centres6_2 = np.mean(centres_2, axis=0)
centres6_3 = np.mean(centres_3, axis=0)
centres6_4 = np.mean(centres_4, axis=0)
centres6_5 = np.mean(centres_5, axis=0)
centres6_6 = np.mean(centres_6, axis=0)

centroids_6.extend((centres6_1, centres6_2, centres6_3, centres6_4, centres6_5, centres6_6))

In [95]:
centre_1 = np.reshape(centres6_1,(-1,44))
centre_2 = np.reshape(centres6_2,(-1,44))
centre_3 = np.reshape(centres6_3,(-1,44))
centre_4 = np.reshape(centres6_4,(-1,44))
centre_5 = np.reshape(centres6_5,(-1,44))
centre_6 = np.reshape(centres6_6,(-1,44))

In [96]:
freq_h6 = Counter(cluster6.labels_)

# Plotting the nodes
plt.suptitle("Hierarchical n = 6")
plt.subplot(3,2,1)
plt.title("Freq --> %.2f%%" %(100*freq_h6[2]/len(whole_data_som[:,1])))
plt.imshow(centre_3, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,2)
plt.title("Freq --> %.2f%%" %(100*freq_h6[4]/len(whole_data_som[:,1])))
plt.imshow(centre_5, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36' ,cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,3)
plt.title("Freq --> %.2f%%" %(100*freq_h6[0]/len(whole_data_som[:,1])))
plt.imshow(centre_1, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,4)
plt.title("Freq --> %.2f%%" %(100*freq_h6[1]/len(whole_data_som[:,1])))
plt.imshow(centre_2, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
#plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,5)
plt.title("Freq --> %.2f%%" %(100*freq_h6[5]/len(whole_data_som[:,1])))
plt.imshow(centre_6, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.subplot(3,2,6)
plt.title("Freq --> %.2f%%" %(100*freq_h6[3]/len(whole_data_som[:,1])))
plt.imshow(centre_4, vmin = -10, vmax = 25, aspect = 'auto', interpolation = 'spline36', cmap = 'nipy_spectral', origin = 'lower')
plt.xticks(np.arange(0,45,5), np.arange(25,91,7.5))
plt.xlabel("Latitude(degrees North)")
plt.yticks(np.arange(0,6,1), reversed(np.arange(250,851,100)))
plt.ylabel("Pressure Levels")
plt.grid(color='w', linestyle='--', linewidth=0.5)
plt.colorbar()
plt.show()

In [164]:
np.asarray(centroids_3).shape

(9, 783)

In [65]:
# correlation matrices

C_2 = np.corrcoef(np.asarray(centroids_2))

In [165]:
C_3 = np.corrcoef(np.asarray(centroids_3))

In [107]:
C_4 = np.corrcoef(np.asarray(centroids_4))

C:\Users\krish\Anaconda3\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\krish\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\krish\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\krish\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\krish\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [118]:
C_6 = np.corrcoef(np.asarray(centroids_6))

In [30]:
import seaborn as sns

In [166]:
plt.figure()
sns.heatmap(C_3, annot = True)
plt.show()

In [100]:
round(np.corrcoef([[1,2],[1,2]])[0,1], 3)

1.0

In [291]:
whole_data_som[:,:].shape

(3680, 783)

In [349]:
check = [1,2]
if check == [1,2]:
    print("works")
else:
    print("doesn't")

works


In [54]:
# Time series 3-SOMs

time_series(som3, 7, year_2003)

2249
2238
2252
2249
2242
2258
2259
2243
2261
2258
2248
2246
2245
2243
2267
2243
2269
2269
2271
2272
2271
2272
2275
2276
2277
2269
2283
2277
2286
2271


In [53]:
def time_series(SOM, time_interval, time_period):
    #Main
    # Getting Frequency of each cluster as a timeseries
    # unit interval of the timeseries, 7 for weekly, 30 for monthly, and 92 for yearly
    # However for seasonal timeseries, I am using a unit interval of 3,5,7(I dont which works the best).
    # time_period refers to the year or years over which the frequency of each cluster event is plotted. 
    #time_interval = 3 
    cluster_1 = []
    cluster_2 = []
    cluster_3 = []
    #cluster_4 = []

    c1, c2 = 0, 0 
    c3 = 0 #, c4 = 0, 0, 0, 0
    tot_1, tot_2 = 0, 0 
    tot_3 = 0 
    #tot_4 = 0, 0, 0, 0
    time = 0
    for day in whole_data_som[time_period,:]:
        node = SOM.winner(day)
        day_pos = np.where(whole_data_som == day)[0][0]
        time += 1
        if node == (0,0):
            c1 += 1
            tot_1 += 1
            
            print(day_pos)
                #f
        elif node == (1,0):
            c2 += 1
            tot_2 += 1
        elif node == (2,0):
            c3 += 1
            tot_3 += 1
        #else:
           # c4 += 1
            #tot_4 += 1
        if time == time_interval:
            cluster_1.append(c1)
            cluster_2.append(c2)
            cluster_3.append(c3)
            #cluster_4.append(c4)
            c1, c2, c3 =0, 0, 0#, c3, c4 = 0, 0, 0, 0
            time = 0

    if time !=0:
        cluster_1.append(c1)
        cluster_2.append(c2)
        cluster_3.append(c3)
        #cluster_4.append(c4)
    years = np.arange(1979, 2019)
    months = np.arange(1, 3681, 30)
    weeks = np.arange(1, 3681, 7)
    total_time_periods = np.arange(1, len(cluster_1)+1)
    JJA = np.arange(1,93, time_interval)
    plt.figure()
    plt.title("Frequency of Cluster 1 events")
    plt.plot(total_time_periods, cluster_1, label = 'Cluster 1')
    #plt.plot(months, cluster_1, label = 'Cluster 1')
    #plt.xlabel("")
    plt.xlabel("June-July-August")
    plt.ylabel("Frequency")
    plt.figure()
    plt.title("Frequency of Cluster 2 events")
    plt.plot(total_time_periods, cluster_2, label = 'Cluster 2')
    #plt.xlabel("Years")
    plt.xlabel("June-July-August")
    plt.ylabel("Frequency")
    

    plt.figure()
    plt.title("Monthly frequency of Cluster 3 events")
    plt.plot(total_time_periods, cluster_3, label = 'Cluster 3')
    #plt.xlabel("Years")
    plt.xlabel("June-July-August")
    plt.ylabel("Frequency")
    plt.show()
    
    return
   

In [ ]:
# There was so much discomfort when I started writing this.

# frequency count
#Here was number of cluster events happening within this time_interval 

time_interval = 7

for i in 


In [41]:
 
    '''
    plt.figure()
    plt.title("Monthly frequency of Cluster 4 events")
    plt.plot(months, cluster_4, label = 'Cluster 4')
    #plt.xlabel("Years")
    plt.xlabel("Summer months(1979-2018)")
    plt.ylabel("Frequency")
    '''
    #plt.legend()
    #plt.show()
    #print("Overall frequencies")
    # Doesn't make so much sense to print as its being taken directly from the codebook.
    
    #calling the persistence for each node
    #print("C1uster_1-yearly segregation")
    #persistence(cluster_1)
    #print("Cluster_2-yearly segregation")
    #persistence(cluster_2)
    #return

In [226]:
def persistence(cluster_node):
    # Segregration of high frequency, low and mid for double jet events.

    # Time interval chosen being years
    # The frequencies are in ascending order from 1979 till 2018
    highs, mediums, lows  =  dict(), dict(), dict()
    high_threshold, medium_threshold = 35, 25

    for i in np.arange(len(cluster_node)):# or 2 depending on which refers to double. (Decide from the graphs)
        yearly_freq = cluster_node[i]
        year = years[i]
        #print(yearly_freq)
        if yearly_freq >= high_threshold:
            highs[year] = yearly_freq
        elif yearly_freq >= medium_threshold and yearly_freq < high_threshold:
            mediums[year] = yearly_freq
        else:
            lows[year] = yearly_freq

    #Results
    print("Years with high freq of double jet events with corresponding frequencies")
    print(highs)

    print("Years with medium freq of double jet events with corresponding frequencies")
    print(mediums)

    print("Years with low freq of double jet events with corresponding frequencies")
    print(lows)
    return

In [ ]:
# Persistence how it works - shows persistence of events in particular years, and over all years.
# Select the year to be analysised

year = all # all or 2003, 2010, 2018 (hopefully, but its also nice to get surprised)

# Next we have cluster analysis done for this year and have nodes to signify each event.

persistence_1, persistence_2 = 0, 0 
persistences_1, persistences_2 = [], []
for pos in np.arange(len(whole_data_som[year,:])):
    
    node = som2.winner(whole_data_som[year[0]+i,:])
    if node == (0,0):
        persistence_1 += 1
        next_node = som2.winner(whole_data_som[year[0]+i+1,:])
         if next_node != (0,0):
            persistences_1.append(persistence_1)
            persistence_1 = 0
        
    else:
        persistence_2 += 1
        next_node = som6.winner(whole_data_som[i+1,:])
        if next_node != (0,1):
            persistences_2.append(persistence_2)
            persistence_2 = 0

print("Average persistence for cluster 1:", np.mean(persistences_1))
print("Average persistence for cluster 2:", np.mean(persistences_2))


# Plotting the progression of persistence of each cluster event along JJA
plt.figure()
plt.suptitle("Persistence of each event along time")
plt.subplot(2,1,1)
plt.title("Cluster 1")
plt.plot(persistences_1)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.subplot(2,1,2)
plt.title("Cluster 2")
plt.plot(persistences_2)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.show()

In [ ]:
# Segregation of double jet events into formation, maintenance, and decay phases

# This might be a tough cookie



In [ ]:
# PLotting the circulation of u and v for hand picked double jet events

# Major steps here are hand picking the events.
    # Which method should I use

# Next is plotting the map. Might have to read about that first.


In [ ]:
# Checking the persistence of the cluster events, and particularly double jet

# Code written for 6 SOM, but conversion to 4 SOM shouldn't be hard.
persistence_1, persistence_2, persistence_3, persistence_4 = 0, 0, 0, 0 
persistence_5, persistence_6 = 0, 0
persistences_1, persistences_2, persistences_3, persistences_4 = [], [], [], []
persistences_5, persistences_6 = [], []
for i in np.arange(len(whole_data_som[:,1])):
    node = som6.winner(whole_data_som[i,:])
    if node == (0,0):
        persistence_1 += 1
        next_node = som6.winner(whole_data_som[i+1,:])
        if next_node != (0,0):
            persistences_1.append(persistence_1)
            persistence_1 = 0
        
    elif node == (0,1):
        persistence_2 += 1
        next_node = som6.winner(whole_data_som[i+1,:])
        if next_node != (0,1):
            persistences_2.append(persistence_2)
            persistence_2 = 0
    
    elif node == (1,0):
        persistence_3 += 1
        next_node = som6.winner(whole_data_som[i+1,:])
        if next_node != (1,0):
            persistences_3.append(persistence_3)
            persistence_3 = 0
    
    elif node == (1,1):
        persistence_4 += 1
        next_node = som6.winner(whole_data_som[i+1,:])
        if next_node != (1,1):
            persistences_4.append(persistence_4)
            persistence_4 = 0
    
    elif node == (2,0):
        persistence_5 += 1
        next_node = som6.winner(whole_data_som[i+1,:])
        if next_node != (2,0):
            persistences_5.append(persistence_5)
            persistence_5 = 0
    
    else:
        persistence_6 += 1
        next_node = som6.winner(whole_data_som[i+1,:])
        if next_node != (2,1):
            persistences_6.append(persistence_6)
            persistence_6 = 0
    
print("Average persistence for cluster 1:", np.mean(persistences_1))
print("Average persistence for cluster 2:", np.mean(persistences_2))
print("Average persistence for cluster 3:", np.mean(persistences_3))
print("Average persistence for cluster 4:", np.mean(persistences_4))
print("Average persistence for cluster 5:", np.mean(persistences_5))
print("Average persistence for cluster 6:", np.mean(persistences_6))

# Plotting the progression of persistence of each cluster event along years
plt.figure()
plt.suptitle("Persistence of each event along time")
plt.subplot(3,2,1)
plt.title("Cluster 1")
plt.plot(persistences_1)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.subplot(3,2,2)
plt.title("Cluster 2")
plt.plot(persistences_2)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.subplot(3,2,3)
plt.title("Cluster 3")
plt.plot(persistences_3)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.subplot(3,2,4)
plt.title("Cluster 4")
plt.plot(persistences_4)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.subplot(3,2,5)
plt.title("Cluster 5")
plt.plot(persistences_5)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.subplot(3,2,6)
plt.title("Cluster 6")
plt.plot(persistences_6)
plt.ylabel("Number of days")
plt.xlabel("Events along years")
plt.show()

In [245]:
#Looping over the whole set of days

# Double jet variable for 4SOM comes from 1st cluster for 6SOM from 3rd cluster

# After updating the dataset and running SOM with seed. First cluster from 2003 is taken as the base double jet event 

# How is the correlation coefficient determined
threshold = 0.90
double_jet = som4.get_weights()[0,0,:]

tot_dj_days = 0 # Total double jet days

# interval is 92 for yearly, 30 for monthly and 7 for weekly time series
time_series_interval = 92
count = 0

dj_days = 0
# Number of double jet events in each interval
djs = []

all_djs_counted = False

for i in np.arange(len(whole_data_som[:,1])):
    count +=1
    
    day = whole_data_som[i,:]
    corr = round(np.corrcoef([day,double_jet])[0,1], 3)
    if corr>threshold:
        tot_dj_days += 1
        dj_days += 1
    if count == time_series_interval:
        count = 0
        djs.append(dj_days)
        dj_days = 0
        if i == len(whole_data_som[:,1])-1:
            all_djs_counted = True
if all_djs_counted == False:
    djs.append(dj_days)
#print(len(djs))
print("Total number of days with Double Jets : ", tot_dj_days)
years = np.arange(1979,2019)
print("Number of time intervals without any double jet event: ", djs.count(0))
# Crude plotting
plt.figure()
plt.title('Frequency of Double Jet events for time interval %d days' %(time_series_interval))
plt.plot(years, djs)
plt.xlabel('Years')
#plt.xlabel("in Weeks")
plt.ylabel('Number of Double Jet events')
#plt.xticks(years)
plt.show()

Total number of days with Double Jets :  324
Number of time intervals without any double jet event:  5


In [246]:
# Checking the persistence and segregating it into development, mantenance and decay periods

dev = []
main = []
dec = []

# These correlation coefficients and arbitary at the moment 
dj_cor = 0.85
range = 0.15
# cluster 3 from 4 SOM 
double_jet = som4.get_weights()[0,0,:]
corr_distribution = []
gap_distribution = []
persistence = 0
persistence_values = []
djs = 0
start = False
gap = 0
for i in np.arange(len(whole_data_som[:,1])):
    day = whole_data_som[i,:]
    corr = round(np.corrcoef([day, double_jet])[0,1], 3)
    corr_distribution.append(corr)
    gap +=1
    if corr >= dj_cor:
        start = True
        gap -=1
        if(persistence == 0):
            gap_distribution.append(gap)
            gap = 0
            #print("i before :", i)
            development = past(i, dj_cor-range)
            #print("i after :", i)
            dev.append(development)
        persistence += 1
    if corr < dj_cor and start:
        start = False
        persistence_values.append(persistence)
        decay = future(i, dj_cor-range)
        dec.append(decay)
        persistence = 0
        djs += 1 

print("FOR Correlation Coefficient of %f" %(dj_cor))        
print("Separate dj's detected :", djs)
print("Corresponding development units:", len(dev))
print("Corresponding decay units:", len(dec))
print("Average persistence detected", round(np.mean(persistence_values),3))
print("Average development phase length", round(np.mean(dev), 3))
print("Average decay phase length", round(np.mean(dec), 3))
print("Number of gaps:", len(gap_distribution))
print("Average gap between double jets", round(np.mean(gap_distribution)))
#Plotting
plt.figure()
plt.title("Persistence of the Double Jets")
plt.plot(np.arange(1,djs+1),persistence_values, label = 'Maintenance')
plt.plot(np.arange(1,djs+1), dev, label = 'Development')
plt.plot(np.arange(1,djs+1), dec, label = 'Decay')
plt.xlabel("Double Jet Event")
plt.ylabel("Number of days")
plt.legend()
plt.figure()
plt.title("Correlation coeff distribution with Double Jets")
y=4
plt.plot(corr_distribution[92*y:92*(y+1)])
plt.xlabel("Time in days")
plt.ylabel("correlation coefficient")
plt.figure()
plt.title("Gap distribution between Double Jets")
plt.plot(gap_distribution)
plt.ylabel("Time in days")
plt.xlabel("DoubleJet events")
plt.show()


FOR Correlation Coefficient of 0.850000
Separate dj's detected : 160
Corresponding development units: 160
Corresponding decay units: 160
Average persistence detected 4.656
Average development phase length 0.988
Average decay phase length 0.944
Number of gaps: 160
Average gap between double jets 18.0


In [168]:
# Look at past days
def past(index, cutoff):
    '''
    Takes in the index and adds any past days if correlation within the cutoff.
    Returns the number of days in the devolopment stage.
    '''
    run = True
    count = 0
    while(run):
        index -= 1
        # Hope that the same varible name would not cause any problems
        day = whole_data_som[index,:]
        corr = round(np.corrcoef([day, double_jet])[0,1], 3)
        if(corr < cutoff):
            run = False
        else:
            count +=1
        return count
    
def future(index, cutoff):
    """
    Looks at future and counts the number of decay days
    """
    run = True
    count = 0
    while(run):
        
        # Hope that the same varible name would not cause any problems
        day = whole_data_som[index,:]
        index += 1
        corr = round(np.corrcoef([day, double_jet])[0,1], 3)
        if(corr < cutoff):
            run = False
        else:
            count +=1
        return count

In [ ]:
for i in np.arange(len(whole_data_som[:,1])):
    count +=1
    
    day = whole_data_som[i,:]
    corr = round(np.corrcoef([day,double_jet])[0,1], 3)
    
    if corr > dj_cor:
        go_back, j = True, i
        go_forward, k = True, i
        #Checks for persistence in the days before it
        while (go_back):
            r_day = whole_data_som[j-1,:]
            r_corr = round(np.corrcoef([r_day,double_jet])[0,1], 3)
            if(r_corr > dj_corr):
                main_c = 
        while (go_forward)
            
    if corr>threshold:
        tot_dj_days += 1
        dj_days += 1
    if count == time_series_interval:
        count = 0
        djs.append(dj_days)
        dj_days = 0
        if i == len(whole_data_som[:,1])-1:
            all_djs_counted = True

In [ ]:
# Printing out which correlated to which the best

# 2 cluster scenario

n = 2
for var in C_2[0:n,:]:
    for 

In [ ]:
# Plotting after taking spatial correlation into account

In [ ]:
# CHecking the correlation of double jet event to different days in the summer
# also from difffernt methods 

# Frequency checker

# Steps get a double jet looking cluster centre
# Decide the correaltion threshold
# Look at days one by one and take count of days it's more.

double_jet =
count = 0

In [8]:
kk = [1, 2, 4, 5, 6, 6, 5]
kk[1:4:2]


[2, 5]

In [76]:
# Counting number of double jet events

c1_persistence, c2_persistence, c3_persistence = 0, 0, 0
c1_count, c2_count, c3_count = 0, 0, 0
c1_persistences, c2_persistences, c3_persistences = [], [], []
c1_s, c2_s, c3_s = [], [], []
time = 0
time_interval = 92
for i in np.arange(len(whole_data_som[:,0])):
    time += 1
    cluster1 = som3.winner(whole_data_som[i,:])
    #print(c1_persistence)
    if (i+1) < len(whole_data_som[:,0]):
        cluster2 = som3.winner(whole_data_som[i+1,:])
    if cluster1 == cluster2:
        
        if cluster1 == (0,0):
            c1_persistence += 1
        elif cluster1 == (1,0):
            c2_persistence += 1
        elif cluster1 == (2,0):
            c3_persistence += 1
        
    else:
        if cluster1 == (0,0):
            c1_persistence += 1
            c1_persistences.append(c1_persistence)
            c1_persistence = 0
            c1_count += 1
        elif cluster1 == (1,0):
            c2_persistence += 1
            c2_persistences.append(c2_persistence)
            c2_persistence = 0
            c2_count += 1
        elif cluster1 == (2,0):
            c3_persistence += 1
            c3_persistences.append(c3_persistence)
            c3_persistence = 0
            c3_count += 1
            
    if time == time_interval:
        time = 0
        c1_s.append(c1_count)
        c2_s.append(c2_count)
        c3_s.append(c3_count)
        c1_count = 0
        c2_count = 0
        c3_count = 0

print("Average Persistence of Cluster 1 events", np.mean(c1_persistences))
print("Average Persistence of Cluster 2 events", np.mean(c2_persistences))
print("Average Persistence of Cluster 3 events", np.mean(c3_persistences))

        
        
        
    


Average Persistence of Cluster 1 events 6.990243902439024
Average Persistence of Cluster 2 events 8.584905660377359
Average Persistence of Cluster 3 events 6.059360730593608


In [77]:
c1_persistences

[2,
 1,
 8,
 12,
 2,
 23,
 16,
 15,
 11,
 12,
 2,
 5,
 10,
 3,
 11,
 5,
 6,
 5,
 6,
 3,
 2,
 4,
 6,
 11,
 11,
 4,
 6,
 10,
 3,
 3,
 14,
 15,
 12,
 2,
 2,
 8,
 3,
 12,
 5,
 2,
 6,
 1,
 8,
 6,
 2,
 2,
 4,
 2,
 3,
 13,
 12,
 9,
 2,
 6,
 21,
 6,
 6,
 1,
 5,
 5,
 7,
 8,
 2,
 1,
 7,
 2,
 38,
 2,
 2,
 3,
 2,
 1,
 12,
 6,
 4,
 7,
 52,
 4,
 16,
 31,
 11,
 1,
 3,
 10,
 3,
 27,
 1,
 1,
 9,
 6,
 13,
 3,
 1,
 1,
 2,
 7,
 7,
 4,
 3,
 1,
 2,
 3,
 6,
 5,
 5,
 10,
 5,
 2,
 11,
 5,
 16,
 1,
 2,
 3,
 10,
 8,
 1,
 3,
 3,
 7,
 3,
 1,
 4,
 26,
 4,
 8,
 2,
 13,
 3,
 5,
 9,
 7,
 3,
 6,
 2,
 4,
 2,
 9,
 1,
 1,
 14,
 11,
 19,
 34,
 3,
 8,
 3,
 2,
 7,
 5,
 1,
 8,
 3,
 2,
 3,
 4,
 2,
 7,
 4,
 2,
 4,
 3,
 3,
 1,
 36,
 9,
 3,
 3,
 4,
 2,
 3,
 10,
 7,
 11,
 2,
 2,
 2,
 7,
 7,
 5,
 4,
 27,
 1,
 4,
 14,
 5,
 2,
 3,
 5,
 2,
 2,
 19,
 2,
 3,
 4,
 3,
 21,
 4,
 9,
 11,
 11,
 16,
 11,
 1,
 28]

In [80]:
plt.figure()
plt.plot(np.arange(1979, 2019), c1_s)
plt.plot(np.arange(1979, 2019), c2_s)
plt.plot(np.arange(1979, 2019), c3_s)
plt.show()

In [479]:
def freq_plot_one(clustering, dataset, years):
    """
    At the moment the clustering is applicable only for soms. Need to modify to include every method.
    """
    count = 0
    fig, ax = plt.subplots()
    plt.title("Event distribution for June-July-August 2010")
    for i in years:
        count +=1
        day = dataset[i,:]
        y = clustering.winner(day)[0]
        x = dates[i]
        if y == 0:
            ax.plot_date(x,'Cluster 1','ro')
        elif y == 1:
            ax.plot_date(x,'Cluster 2','bo')
        else:
            ax.plot_date(x,'Cluster 3','go')
    fig.autofmt_xdate()
    ax.fmt_xdata = mdates.DateFormatter('%Y-%M-%d')
    ax.set_xlim(dates[years[0]],dates[years[-1]])
    locations = mdates.WeekdayLocator(mdates.MO)
    ax.xaxis.set_major_locator(locations)
    plt.xlabel("Date")
    plt.ylabel("Event Class")
    plt.show()
    print(count)
    return

def freq_plot_two(clustering, dataset, time_interval, years):
    count = 0
    reset = 0
    c1_series = []
    c2_series = []
    c3_series = []
    no_f_c1 = 0 
    no_f_c2 = 0 
    no_f_c3 = 0
    time_intervals = []
    for i in years:
        count += 1
        reset += 1
        day = dataset[i,:]
        cluster = clustering.winner(day)[0]
        if cluster == 0:
            no_f_c1 += 1
        elif cluster == 1:
            no_f_c2 += 1
        elif cluster == 2:
            no_f_c3 += 1
        
        if reset == time_interval:
            
            reset = 0
            c1_series.append(no_f_c1)
            no_f_c1 = 0
            c2_series.append(no_f_c2)
            no_f_c2 = 0
            c3_series.append(no_f_c3)
            no_f_c3 = 0
        
        
    if reset != 0:
        reset = 0
        c1_series.append(no_f_c1)
        no_f_c1 = 0
        c2_series.append(no_f_c2)
        no_f_c2 = 0
        c3_series.append(no_f_c3)
        no_f_c3 = 0
    x = np.concatenate((dates[years[0]:years[-1]:time_interval],dates[years[-1]]), axis = None)
    fig, ax = plt.subplots()
    plt.title("Cluster 1 weekly timeseries for June-July-August 2018")
    #locations = mdates.DayLocator(bymonthday=dates[years[0]:years[-1]], interval=time_interval)
    ax.plot_date(x, c1_series,'o-', label = 'Cluster 1')
    fig.autofmt_xdate()
    ax.set_xlim(dates[years[0]],dates[years[-1]])
    locations = mdates.WeekdayLocator(mdates.MO)
    ax.xaxis.set_major_locator(locations)
    ax.fmt_xdata = mdates.DateFormatter('%Y-%M-%d')
    plt.xlabel('Date')
    plt.ylabel('Number of days')
    
    fig, ax = plt.subplots()
    plt.title("Cluster 2 weekly timeseries for June-July-August 2018")
    #locations = mdates.DayLocator(bymonthday=dates[years[0]:years[-1]], interval=time_interval)
    ax.plot_date(x, c2_series,'o-', label = 'Cluster 2')
    plt.xlabel('Date')
    plt.ylabel('Number of days')
    fig.autofmt_xdate()
    ax.set_xlim(dates[years[0]],dates[years[-1]])
    locations = mdates.WeekdayLocator(mdates.MO)
    ax.xaxis.set_major_locator(locations)
    ax.fmt_xdata = mdates.DateFormatter('%Y-%M-%d')
    
    fig, ax = plt.subplots()
    plt.title("Cluster 3 weekly timeseries for June-July-August 2018")
    #locations = mdates.DayLocator(bymonthday=dates[years[0]:years[-1]], interval=time_interval)
    ax.plot_date(x, c3_series,'o-', label = 'Cluster 3')
    fig.autofmt_xdate()
    ax.set_xlim(dates[years[0]],dates[years[-1]])
    locations = mdates.WeekdayLocator(mdates.MO)
    ax.xaxis.set_major_locator(locations)
    ax.fmt_xdata = mdates.DateFormatter('%Y-%M-%d')
    #ax.legend()#loc='upper left', frameon=False)
    plt.xlabel('Date')
    plt.ylabel('Number of days')
    
    plt.show()
    print(count)
    return
            
def freq_plot_three(clustering, dataset, years):
    pass


In [481]:
# Time Series AKA Frequency plots

# Template : Summer of 2003

freq_plot_one(som3, whole_data_som, year_2010)
#freq_plot_two(som3, whole_data_som, 7, year_2018)
#freq_plot_three()

92


In [252]:
dates[year_2003[-1]]

numpy.datetime64('2003-08-31T09:00:00.000000000')

In [31]:
def jetmosaic_plot_soms(dataset):
    """
    Made for SOMS
    """
    years = np.arange(1979, 2019)
    total_years = len(years)
    total_summer_days = len(year_2003)
    jet = np.empty((total_years,total_summer_days))
    for i in np.arange(len(dataset[:,1])):
        day = dataset[i,:]
        cluster_som = som3.winner(day)[0]
        y_coord, x_coord = i%total_summer_days, int(i/total_summer_days)
        #print(x_coord)
        jet[x_coord, y_coord] = cluster_som
    #print(jet.shape)
    summer_days = dates[year_2003[0]:year_2003[-1]+1]
    col_names = pd.to_datetime(summer_days).strftime("%d-%m")
    print(len(summer_days))
    df = pd.DataFrame(jet, index = years, columns = col_names)
    cmap = plt.cm.get_cmap('GnBu_r', 3)
    #cmap = colors.ListedColormap(['khaki', 'lightskyblue', 'lavenderblush'],)
    plt.figure()
    ax = sns.heatmap(df, annot = False, cmap=cmap, linewidths=.5, linecolor='lightgray')
    ax.set(xlabel="Date(Day/Month)", ylabel="Years (1979-2018)", title="Daily Event Classification(SOM)")
    #plt.colorbar(ax, ticks=["Cluster 1","Cluster 2","Cluster 3"])
    #ax.set_yticklabels(np.arange(1979,2019),rotation = 0)
    #ax.set_xticklabels(dates[year_2003[0]:year_2003[-1]])
    #locations = mdates.WeekdayLocator(interval = 2)
    #ax.xaxis.set_major_locator(locations)
    #ax.fmt_xdata = mdates.DateFormatter('%M-%d')
    #print(df)
    # Manually specify colorbar labelling after it's been generated
    colorbar = ax.collections[0].colorbar
    colorbar.set_ticks([0.25, 1, 1.75])
    colorbar.set_ticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])
    plt.show()
    return df
        

In [490]:
def jetmosaic_plot_kmeans(dataset):
    """
    Made for Kmeans 
    """
    years = np.arange(1979, 2019)
    total_years = len(years)
    total_summer_days = len(year_2003)
    jet = np.empty((total_years,total_summer_days))
    clusters = kmeans3.predict(dataset)
    for i in np.arange(len(dataset[:,1])):
        day = dataset[i,:]
        cluster_kmeans = clusters[i]
        if cluster_kmeans == 0:
            cluster_kmeans = 1
        elif cluster_kmeans == 1:
            cluster_kmeans = 0
        else:
            cluster_kmeans = 2
        y_coord, x_coord = i%total_summer_days, int(i/total_summer_days)
        #print(x_coord)
        jet[x_coord, y_coord] = cluster_kmeans
    #print(jet.shape)
    summer_days = dates[year_2003[0]:year_2003[-1]+1]
    col_names = pd.to_datetime(summer_days).strftime("%d-%m")
    print(len(summer_days))
    df = pd.DataFrame(jet, index = years, columns = col_names)
    cmap = plt.cm.get_cmap('GnBu_r', 3)
    #cmap = colors.ListedColormap(['khaki', 'lightskyblue', 'lavenderblush'],)
    plt.figure()
    ax = sns.heatmap(df, annot = False, cmap=cmap, linewidths=.5, linecolor='lightgray')
    ax.set(xlabel="Date(Day/Month)", ylabel="Years (1979-2018)", title="Daily Event Classification(Kmeans)")
    #plt.colorbar(ax, ticks=["Cluster 1","Cluster 2","Cluster 3"])
    #ax.set_yticklabels(np.arange(1979,2019),rotation = 0)
    #ax.set_xticklabels(dates[year_2003[0]:year_2003[-1]])
    #locations = mdates.WeekdayLocator(interval = 2)
    #ax.xaxis.set_major_locator(locations)
    #ax.fmt_xdata = mdates.DateFormatter('%M-%d')
    #print(df)
    # Manually specify colorbar labelling after it's been generated
    colorbar = ax.collections[0].colorbar
    colorbar.set_ticks([0.25, 1, 1.75])
    colorbar.set_ticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])
    plt.show()
    return df

In [491]:
def jetmosaic_plot_hierarchical(dataset):
    """
    Made for Kmeans 
    """
    years = np.arange(1979, 2019)
    total_years = len(years)
    total_summer_days = len(year_2003)
    jet = np.empty((total_years,total_summer_days))
    clusters = cluster3.labels_
    for i in np.arange(len(dataset[:,1])):
        day = dataset[i,:]
        cluster_kmeans = clusters[i]
        if cluster_kmeans == 0:
            cluster_kmeans = 1
        elif cluster_kmeans == 1:
            cluster_kmeans = 0
        else:
            cluster_kmeans = 2
        y_coord, x_coord = i%total_summer_days, int(i/total_summer_days)
        #print(x_coord)
        jet[x_coord, y_coord] = cluster_kmeans
    #print(jet.shape)
    summer_days = dates[year_2003[0]:year_2003[-1]+1]
    col_names = pd.to_datetime(summer_days).strftime("%d-%m")
    print(len(summer_days))
    df = pd.DataFrame(jet, index = years, columns = col_names)
    cmap = plt.cm.get_cmap('GnBu_r', 3)
    #cmap = colors.ListedColormap(['khaki', 'lightskyblue', 'lavenderblush'],)
    plt.figure()
    ax = sns.heatmap(df, annot = False, cmap=cmap, linewidths=.5, linecolor='lightgray')
    ax.set(xlabel="Date(Day/Month)", ylabel="Years (1979-2018)", title="Daily Event Classification(Hierarchical)")
    #plt.colorbar(ax, ticks=["Cluster 1","Cluster 2","Cluster 3"])
    #ax.set_yticklabels(np.arange(1979,2019),rotation = 0)
    #ax.set_xticklabels(dates[year_2003[0]:year_2003[-1]])
    #locations = mdates.WeekdayLocator(interval = 2)
    #ax.xaxis.set_major_locator(locations)
    #ax.fmt_xdata = mdates.DateFormatter('%M-%d')
    #print(df)
    # Manually specify colorbar labelling after it's been generated
    colorbar = ax.collections[0].colorbar
    colorbar.set_ticks([0.25, 1, 1.75])
    colorbar.set_ticklabels(['Cluster 1', 'Cluster 2', 'Cluster 3'])
    plt.show()
    return df

In [31]:
# Count indices of c1 and c3 dominated dates in July and August

def get_c1_c3_indices(dataset, June=True):
    """
    Returns indices of days dominated by c1 and c3 in July
    """
    c1_indices = []
    c3_indices = []
    #years = np.arange(1979, 2019)
    #total_years = len(years)
    #total_summer_days = len(year_2003)
    #jet = np.empty((total_years,total_summer_days))
    for i in np.arange(len(dataset[:,1])):
        day = dataset[i,:]
        cluster_som = som3.winner(day)[0]
        if June == True:
            if cluster_som == 0:
                c1_indices.append(i)
            elif cluster_som == 2:
                c3_indices.append(i)
        else:
            if cluster_som == 0 and (i%92)>29:
                c1_indices.append(i)
            elif cluster_som == 2 and (i%92)>29:
                c3_indices.append(i)
    return c1_indices, c3_indices
    

In [32]:
#daily_dataframe = jetmosaic_plot_soms(whole_data_som)
#jetmosaic_plot_kmeans(whole_data_som)
#jetmosaic_plot_hierarchical(whole_data_som)
c1_days, c3_days = get_c1_c3_indices(whole_data_som)
c1_days_wo_jun, c3_days_wo_jun = get_c1_c3_indices(whole_data_som, June=False)

In [36]:
# Passing c1_days and c3_days to other scripts
%store c1_days 
%store c3_days
%store c1_days_wo_jun
%store c3_days_wo_jun

Stored 'c1_days' (list)
Stored 'c3_days' (list)
Stored 'c1_days_wo_jun' (list)
Stored 'c3_days_wo_jun' (list)


In [35]:
print(len(c1_days),len(c3_days),len(whole_data_som[:,1]))
print(len(c1_days_wo_jun), len(c3_days_wo_jun))

1443 1327 3680
1307 1103


In [33]:
daily_dataframe
july_dataframe = daily_dataframe.iloc[:,30:61].transpose().copy()
august_dataframe = daily_dataframe.iloc[:,61:92].transpose().copy()
july_dataframe[2010].value_counts()

0.0    31
Name: 2010, dtype: int64

In [34]:


# Putting values in a numoy array
years = np.arange(1979, 2019)
total_years = len(years)
July = np.empty((total_years, 2))
August = np.empty((total_years, 2))
for year in years:
    print(year)
    if year == 1994 or year == 2006:
        July[year-1979, 0] = july_dataframe[year].value_counts()[0]
        August[year-1979, 0] = august_dataframe[year].value_counts()[0]
        July[year-1979, 1] = july_dataframe[year].value_counts()[2]
        August[year-1979, 1] = 0
    elif year == 2010:
        July[year-1979, 0] = july_dataframe[year].value_counts()[0]
        August[year-1979, 0] = august_dataframe[year].value_counts()[0]
        July[year-1979, 1] = 0
        August[year-1979, 1] = august_dataframe[year].value_counts()[2]
    else:
        July[year-1979, 0] = july_dataframe[year].value_counts()[0]
        August[year-1979, 0] = august_dataframe[year].value_counts()[0]
        July[year-1979, 1] = july_dataframe[year].value_counts()[2]
        August[year-1979, 1] = august_dataframe[year].value_counts()[2]
col_names = ['Cluster 1', 'Cluster 3']
df_july = pd.DataFrame(July, index = years, columns = col_names)
df_august = pd.DataFrame(August, index = years, columns = col_names)    

1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018


In [35]:
delta = 0
conditions = [(df_july['Cluster 1'] > df_july['Cluster 3']+delta), (df_july['Cluster 1']+delta < df_july['Cluster 3']), (df_july['Cluster 1'] == df_july['Cluster 3'])]
choices = ['Cluster 1', 'Cluster 3', 'Neither']
df_july['Dominant'] = np.select(conditions, choices, default=np.nan)
df_july['Dominant'].value_counts()

Cluster 1    26
Cluster 3    13
Neither       1
Name: Dominant, dtype: int64

In [36]:
conditions = [(df_august['Cluster 1'] > df_august['Cluster 3']+delta), (df_august['Cluster 1']+delta < df_august['Cluster 3']), (df_august['Cluster 1'] == df_august['Cluster 3'])]
choices = ['Cluster 1', 'Cluster 3', 'Neither']
df_august['Dominant'] = np.select(conditions, choices, default=np.nan)
df_august['Dominant'].value_counts()

Cluster 1    21
Cluster 3    19
Name: Dominant, dtype: int64

In [37]:
July_c1, July_c3, July_neither = [], [], []
August_c1, August_c3, August_neither = [], [], []
for year in years:
    if df_july.loc[year,'Dominant'] == 'Cluster 1':
        July_c1.append(year)
    elif df_july.loc[year,'Dominant'] == 'Cluster 3':
        July_c3.append(year)
    else:
        July_neither.append(year)

#   Similarly for August        
    if df_august.loc[year,'Dominant'] == 'Cluster 1':
        August_c1.append(year)
    elif df_august.loc[year,'Dominant'] == 'Cluster 3':
        August_c3.append(year)
    else:
        August_neither.append(year)
print(July_c1)
print(August_c1)

[1981, 1982, 1984, 1986, 1987, 1989, 1990, 1991, 1992, 1993, 1994, 1996, 1997, 1998, 2000, 2001, 2003, 2004, 2005, 2006, 2010, 2013, 2014, 2016, 2017, 2018]
[1980, 1981, 1982, 1984, 1986, 1989, 1991, 1992, 1994, 1995, 1997, 2003, 2004, 2005, 2006, 2008, 2010, 2012, 2013, 2016, 2017]


In [38]:
summer_days = dates[year_2003[0]:year_2003[-1]+1]
days = pd.to_datetime(summer_days, format = "%d/%m")
days[0]

Timestamp('2003-06-01 09:00:00')

In [39]:
df_july

,Cluster 1,Cluster 3,Dominant
1979,11.0,13.0,Cluster 3
1980,10.0,21.0,Cluster 3
1981,16.0,15.0,Cluster 1
1982,20.0,8.0,Cluster 1
1983,11.0,16.0,Cluster 3
1984,21.0,10.0,Cluster 1
1985,3.0,25.0,Cluster 3
1986,19.0,11.0,Cluster 1
1987,15.0,14.0,Cluster 1
1988,14.0,16.0,Cluster 3


In [40]:
df_july_sorted = df_july.sort_values(["Cluster 1", "Cluster 3"], ascending = [True, False])
july_c3_dominated = np.sort(df_july_sorted.index[0:13].to_numpy())
july_c1_dominated = np.sort(df_july_sorted.index[-13:].to_numpy())
july_neither_dominated = np.sort(df_july_sorted.index[13:-13].to_numpy())
print(len(july_c3_dominated), len(july_c1_dominated), len(july_neither_dominated))
print(july_c3_dominated)
%store july_c3_dominated 
%store july_c1_dominated 
%store july_neither_dominated

13 13 14
[1979 1980 1983 1985 1988 1995 1999 2007 2008 2009 2011 2012 2015]
Stored 'july_c3_dominated' (ndarray)
Stored 'july_c1_dominated' (ndarray)
Stored 'july_neither_dominated' (ndarray)


In [41]:
df_august_sorted = df_august.sort_values(["Cluster 1", "Cluster 3"], ascending = [True, False])
august_c3_dominated = np.sort(df_august_sorted.index[0:13].to_numpy())
august_c1_dominated = np.sort(df_august_sorted.index[-13:].to_numpy())
august_neither_dominated = np.sort(df_august_sorted.index[13:-13].to_numpy())
print(len(august_c3_dominated), len(august_c1_dominated), len(august_neither_dominated))
print(august_c3_dominated)
%store august_c3_dominated 
%store august_c1_dominated
%store august_neither_dominated

13 13 14
[1983 1988 1990 1993 1998 1999 2002 2007 2009 2011 2014 2015 2018]
Stored 'august_c3_dominated' (ndarray)
Stored 'august_c1_dominated' (ndarray)
Stored 'august_neither_dominated' (ndarray)


In [8]:
august_c1_dominated

NameError: name 'august_c1_dominated' is not defined

In [4]:
"""
Let's categorize July and August for each year(1979- 2018) to c1, c2 or c3 and in between
Terminology 
1 -->  c2
2 --> c3
3 --> in middle
0 --> c1
"""

for 


SyntaxError: invalid syntax (<ipython-input-4-025272f7a309>, line 10)

In [5]:
import cdsapi

In [7]:
import download_ERA5_API 

ModuleNotFoundError: No module named 'download_ERA5_API'